In [8]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy.special import factorial
from scipy.stats import poisson
import math
from matplotlib import animation, rc

In [9]:
def meshgen(nx,ny,xmin,xmax,ymin,ymax):
# return a mesh of size (xmax-xmin)x(ymax-ymin) divided in nx x ny
    mesh=np.zeros((nx*ny,2))
    xstep=(xmax-xmin)/(nx-1)
    ystep=(ymax-ymin)/(ny-1)
    for x in range(nx):
        for y in range(ny):
            mesh[y*nx+x,0]=x*xstep
            mesh[y*nx+x,1]=y*ystep
    return mesh

In [10]:
def initTF(Initial_condition_promoter,Initial_condition_enhancer,Number_promoter,Number_enhancer,Number_Nuclear_protein,mesh,nx,ny,xmin,xmax,ymin,ymax):
# place randomly Number_Nuclear_protein nuclear protein in the mesh on places not occupied by either a promoter or an enhancer
# return a vector containing the node where each
    Node_nuclear_protein=-1*np.ones(Number_Nuclear_protein)
    Temporary_attributed_node=-1
    if Number_Nuclear_protein+Number_promoter+Number_enhancer<(nx*ny)+1:
        for i in range(Number_Nuclear_protein):
            
            while (Temporary_attributed_node*np.ones(Number_Nuclear_protein)==Node_nuclear_protein).any() or (Temporary_attributed_node*np.ones(Number_enhancer)==Initial_condition_enhancer).any() or (Temporary_attributed_node*np.ones(Number_promoter)==Initial_condition_promoter).any():
                xinitcont=np.random.uniform(low=xmin, high=xmax, size=None)
                yinitcont=np.random.uniform(low=ymin, high=ymax, size=None)
                xdif=np.abs(np.linspace(xmin,xmax,nx)-xinitcont)
                xinitdisc=np.argmin(xdif)
                ydif=np.abs(np.linspace(ymin,ymax,ny)-yinitcont)
                yinitdisc=np.argmin(ydif)
                Temporary_attributed_node=int(yinitdisc*nx+xinitdisc)
            Node_nuclear_protein[i]=int(yinitdisc*nx+xinitdisc)
    else:
        print("too much tf")
    return Node_nuclear_protein

In [11]:
def genenextto(Current_position_nuclear_protein,Current_position_enhancer,Current_position_promoter,nx,ny,Number_Nuclear_protein,Number_enhancer,Number_promoter):
#genenextto generate arrays containing the neighbor element of each element as well as their type as function of te position of all molecules

#return Type_next_to_nuclear_protein,Molecule_next_to_nuclear_protein,Type_next_to_promoter,Molecule_next_to_promoter,Type_next_to_enhancer,Molcule_next_to_enhancer

#Type_next_to_i represent the molecule type next to the species i defined as followed
#0 is empty node
#1 is a nuclear protein
#2 is a wall
#3 is an enhancer binding site
#4 is the promoter binding site
#Type_next_to_i is of size number_of_ix4 to define a type in each direction per index which we define as following:
#0 is under
#1 is on the left
#2 is above
#3 is on the right
#Molecule_next_to_i contains the number cheracterizing the molecule next to the considered molecule i 
# knowing that the molecule type is contained in Type_next_to_i
    
    Type_next_to_nuclear_protein=np.zeros((Number_Nuclear_protein,4),int)
    Type_next_to_enhancer=np.zeros((Number_enhancer,4),int)
    Type_next_to_promoter=np.zeros((Number_promoter,4),int)
    Molecule_next_to_nuclear_protein=-1*np.ones((Number_Nuclear_protein,4),int)
    Molcule_next_to_enhancer=-1*np.ones((Number_enhancer,4),int)
    Molecule_next_to_promoter=-1*np.ones((Number_promoter,4),int)
    for i in range(Number_Nuclear_protein):
        #a nuclear protein is next to nuclear protein
        for j in range(Number_Nuclear_protein):
            if (Current_position_nuclear_protein[i]-Current_position_nuclear_protein[j])==-nx:
                Type_next_to_nuclear_protein[i][0]=1
                Molecule_next_to_nuclear_protein[i][0]=j
            elif (Current_position_nuclear_protein[i]-Current_position_nuclear_protein[j])==nx:
                Type_next_to_nuclear_protein[i][2]=1
                Molecule_next_to_nuclear_protein[i][2]=j
            elif (Current_position_nuclear_protein[i]-Current_position_nuclear_protein[j])==-1 and Current_position_nuclear_protein[i]%nx!=nx-1:
                Type_next_to_nuclear_protein[i][1]=1
                Molecule_next_to_nuclear_protein[i][1]=j
            elif (Current_position_nuclear_protein[i]-Current_position_nuclear_protein[j])==1 and Current_position_nuclear_protein[i]%nx!=0:
                Type_next_to_nuclear_protein[i][3]=1
                Molecule_next_to_nuclear_protein[i][3]=j
        #an Enhancer is next to nuclear protein
        for p in range(Number_enhancer):
            if (Current_position_nuclear_protein[i]-Current_position_enhancer[p])==-nx:
                Type_next_to_nuclear_protein[i][0]=3
                Molecule_next_to_nuclear_protein[i][0]=p
            elif (Current_position_nuclear_protein[i]-Current_position_enhancer[p])==nx:
                Type_next_to_nuclear_protein[i][2]=3
                Molecule_next_to_nuclear_protein[i][2]=p
            elif (Current_position_nuclear_protein[i]-Current_position_enhancer[p])==-1 and Current_position_nuclear_protein[i]%nx!=nx-1:
                Type_next_to_nuclear_protein[i][1]=3
                Molecule_next_to_nuclear_protein[i][1]=p
            elif (Current_position_nuclear_protein[i]-Current_position_enhancer[p])==1 and Current_position_nuclear_protein[i]%nx!=0:
                Type_next_to_nuclear_protein[i][3]=3
                Molecule_next_to_nuclear_protein[i][3]=p
        #a promoter is next to nuclear protein
        for h in range(Number_promoter):
            if (Current_position_nuclear_protein[i]-Current_position_promoter[h])==-nx:
                Type_next_to_nuclear_protein[i][0]=4
                Molecule_next_to_nuclear_protein[i][0]=h
            elif (Current_position_nuclear_protein[i]-Current_position_promoter[h])==nx:
                Type_next_to_nuclear_protein[i][2]=4
                Molecule_next_to_nuclear_protein[i][2]=h
            elif (Current_position_nuclear_protein[i]-Current_position_promoter[h])==-1 and Current_position_nuclear_protein[i]%nx!=nx-1:
                Type_next_to_nuclear_protein[i][1]=4
                Molecule_next_to_nuclear_protein[i][1]=h
            elif (Current_position_nuclear_protein[i]-Current_position_promoter[h])==1 and Current_position_nuclear_protein[i]%nx!=0:
                Type_next_to_nuclear_protein[i][3]=4
                Molecule_next_to_nuclear_protein[i][3]=h
        #a wall is next to nuclear protein
        if Current_position_nuclear_protein[i]//nx==ny-1:
            Type_next_to_nuclear_protein[i,0]=2
        if Current_position_nuclear_protein[i]%nx==nx-1:
            Type_next_to_nuclear_protein[i,1]=2
        if Current_position_nuclear_protein[i]//nx==0:
            Type_next_to_nuclear_protein[i,2]=2
        if Current_position_nuclear_protein[i]%nx==0:
            Type_next_to_nuclear_protein[i,3]=2 
    for i in range(Number_promoter):
        #a nuclear protein is next to Prom
        for j in range(Number_Nuclear_protein):
            if (Current_position_promoter[i]-Current_position_nuclear_protein[j])==-nx:
                Type_next_to_promoter[i][0]=1
                Molecule_next_to_promoter[i][0]=j
            elif (Current_position_promoter[i]-Current_position_nuclear_protein[j])==nx:
                Type_next_to_promoter[i][2]=1
                Molecule_next_to_promoter[i][2]=j
            elif (Current_position_promoter[i]-Current_position_nuclear_protein[j])==-1 and Current_position_promoter[i]%nx!=nx-1:
                Type_next_to_promoter[i][1]=1
                Molecule_next_to_promoter[i][1]=j
            elif (Current_position_promoter[i]-Current_position_nuclear_protein[j])==1 and Current_position_promoter[i]%nx!=0:
                Type_next_to_promoter[i][3]=1
                Molecule_next_to_promoter[i][3]=j
        #an Enhancer is next to Prom
        for p in range(Number_enhancer):
            if (Current_position_promoter[i]-Current_position_enhancer[p])==-nx:
                Type_next_to_promoter[i][0]=3
                Molecule_next_to_promoter[i][0]=p
            elif (Current_position_promoter[i]-Current_position_enhancer[p])==nx:
                Type_next_to_promoter[i][2]=3
                Molecule_next_to_promoter[i][2]=p
            elif (Current_position_promoter[i]-Current_position_enhancer[p])==-1 and Current_position_promoter[i]%nx!=nx-1:
                Type_next_to_promoter[i][1]=3
                Molecule_next_to_promoter[i][1]=p
            elif (Current_position_promoter[i]-Current_position_enhancer[p])==1 and Current_position_promoter[i]%nx!=0:
                Type_next_to_promoter[i][3]=3
                Molecule_next_to_promoter[i][3]=p
        #a promoter is next to prom
        for h in range(Number_promoter):
            if (Current_position_promoter[i]-Current_position_promoter[h])==-nx:
                Type_next_to_promoter[i][0]=4
                Molecule_next_to_promoter[i][0]=h
            elif (Current_position_promoter[i]-Current_position_promoter[h])==nx:
                Type_next_to_promoter[i][2]=4
                Molecule_next_to_promoter[i][2]=h
            elif (Current_position_promoter[i]-Current_position_promoter[h])==-1 and Current_position_promoter[i]%nx!=nx-1:
                Type_next_to_promoter[i][1]=4
                Molecule_next_to_promoter[i][1]=h
            elif (Current_position_promoter[i]-Current_position_promoter[h])==1 and Current_position_promoter[i]%nx!=0:
                Type_next_to_promoter[i][3]=4
                Molecule_next_to_promoter[i][3]=h
        if Current_position_promoter[i]//nx==ny-1:
            Type_next_to_promoter[i,0]=2
        if Current_position_promoter[i]%nx==nx-1:
            Type_next_to_promoter[i,1]=2
        if Current_position_promoter[i]//nx==0:
            Type_next_to_promoter[i,2]=2
        if Current_position_promoter[i]%nx==0:
            Type_next_to_promoter[i,3]=2
            
    for i in range(Number_enhancer):
        #a nuclear protein is next to Enh
        for j in range(Number_Nuclear_protein):
            if (Current_position_enhancer[i]-Current_position_nuclear_protein[j])==-nx:
                Type_next_to_enhancer[i][0]=1
                Molcule_next_to_enhancer[i][0]=j
            elif (Current_position_enhancer[i]-Current_position_nuclear_protein[j])==nx:
                Type_next_to_enhancer[i][2]=1
                Molcule_next_to_enhancer[i][2]=j
            elif (Current_position_enhancer[i]-Current_position_nuclear_protein[j])==-1 and Current_position_enhancer[i]%nx!=nx-1:
                Type_next_to_enhancer[i][1]=1
                Molcule_next_to_enhancer[i][1]=j
            elif (Current_position_enhancer[i]-Current_position_nuclear_protein[j])==1 and Current_position_enhancer[i]%nx!=0:
                Type_next_to_enhancer[i][3]=1
                Molcule_next_to_enhancer[i][3]=j
        #an Enhancer is next to Prom
        for p in range(Number_enhancer):
            if (Current_position_enhancer[i]-Current_position_enhancer[p])==-nx:
                Type_next_to_enhancer[i][0]=3
                Molcule_next_to_enhancer[i][0]=p
            elif (Current_position_enhancer[i]-Current_position_enhancer[p])==nx:
                Type_next_to_enhancer[i][2]=3
                Molcule_next_to_enhancer[i][2]=p
            elif (Current_position_enhancer[i]-Current_position_enhancer[p])==-1 and Current_position_enhancer[i]%nx!=nx-1:
                Type_next_to_enhancer[i][1]=3
                Molcule_next_to_enhancer[i][1]=p
            elif (Current_position_enhancer[i]-Current_position_enhancer[p])==1 and Current_position_enhancer[i]%nx!=0:
                Type_next_to_enhancer[i][3]=3
                Molcule_next_to_enhancer[i][3]=p
        #a promoter is next to prom
        for h in range(Number_promoter):
            if (Current_position_enhancer[i]-Current_position_promoter[h])==-nx:
                Type_next_to_enhancer[i][0]=4
                Molcule_next_to_enhancer[i][0]=h
            elif (Current_position_enhancer[i]-Current_position_promoter[h])==nx:
                Type_next_to_enhancer[i][2]=4
                Molcule_next_to_enhancer[i][2]=h
            elif (Current_position_enhancer[i]-Current_position_promoter[h])==-1 and Current_position_enhancer[i]%nx!=nx-1:
                Type_next_to_enhancer[i][1]=4
                Molcule_next_to_enhancer[i][1]=h
            elif (Current_position_enhancer[i]-Current_position_promoter[h])==1 and Current_position_enhancer[i]%nx!=0:
                Type_next_to_enhancer[i][3]=4
                Molcule_next_to_enhancer[i][3]=h
        if Current_position_enhancer[i]//nx==ny-1:
            Type_next_to_enhancer[i,0]=2
        if Current_position_enhancer[i]%nx==nx-1:
            Type_next_to_enhancer[i,1]=2
        if Current_position_enhancer[i]//nx==0:
            Type_next_to_enhancer[i,2]=2
        if Current_position_enhancer[i]%nx==0:
            Type_next_to_enhancer[i,3]=2
            
    return Type_next_to_nuclear_protein,Molecule_next_to_nuclear_protein,Type_next_to_promoter,Molecule_next_to_promoter,Type_next_to_enhancer,Molcule_next_to_enhancer

In [12]:
def coorD_nuclear_protein(Node_reference_number_of_molecule,mesh,Number_of_molecule,Number_step):
# return the coordinates of the molecule as functiun of their node in the mesh
    Coordinates_molecule=np.zeros((Number_step,Number_of_molecule,2))
    for i in range(Number_step):
        for j in range(Number_of_molecule):
            Coordinates_molecule[i,j,:]=mesh[int(Node_reference_number_of_molecule[i,j]),:]
    return Coordinates_molecule

In [13]:
def probagenerator(PolII_per_free_link,k_PolII_on_promoter,k_PolII_in_aggregate,k_PolII_out_of_aggregate,Number_of_PolII_in_aggregate,Number_promoter,Number_enhancer,k_link_nuclear_protein_promoter,k_link_nuclear_protein_enhancer,k_unbinding_nuclear_protein_promoter,k_unbinding_nuclear_protein_enhancer,D_enhancer,D_promoter,Number_of_nuclear_protein,Delta_mRNA,mRNA,Current_position_nuclear_protein,Current_position_promoter,Current_position_enhancer,nx,ny,Number_of_binding_of_nuclear_protein,D_nuclear_protein,Molecule_bound_to_nuclear_protein,k_bind_nuclear_protein_to_nuclear_protein,f,Molecule_bound_to_prom,Number_of_binding_of_promoter,Molecule_next_to_enhancer,Number_of_binding_of_enhancer,k_unbinding_nuclear_protein_nuclear_protein,Number_of_aggregate,Nuclear_protein_in_aggregate,Promoter_in_aggregate,Enhancer_in_aggregate,Number_of_nuclear_protein_in_aggregate,Number_of_promoter_in_aggregate,Number_of_enhancer_in_aggregate):
#compute a time for all possible reaction as defined in the article


    Time_to_link_nuclear_protein_to_nuclear_protein=1000*np.ones((Number_of_nuclear_protein,Number_of_nuclear_protein))
    Time_to_link_nuclear_protein_to_enhancer=1000*np.ones((Number_of_nuclear_protein,Number_enhancer))
    Time_to_link_nuclear_protein_to_promoter=1000*np.ones((Number_of_nuclear_protein,Number_promoter))
    
    Time_to_unbind_nuclear_protein_to_nuclear_protein=1000*np.ones((Number_of_nuclear_protein,Number_of_nuclear_protein))
    Time_to_unbind_nuclear_protein_to_enhancer=1000*np.ones((Number_of_nuclear_protein,Number_enhancer))
    Time_to_unbind_nuclear_protein_to_promoter=1000*np.ones((Number_of_nuclear_protein,Number_promoter))
    
    Time_to_diffuse_nuclear_protein=1000*np.ones((Number_of_nuclear_protein,4))
    Time_to_diffuse_promoter=1000*np.ones((Number_promoter,4))
    Time_to_diffuse_enhancer=1000*np.ones((Number_enhancer,4))
    
    Time_to_bind_polII_to_aggregate=1000*np.ones((Number_of_nuclear_protein))
    Time_to_unbind_polII_from_aggregate=1000*np.ones((Number_of_nuclear_protein))
    
    
    Time_to_degrade_mRNA=1000
    
    Time_to_diffuse_aggregate=1000*np.ones((Number_of_nuclear_protein,4))
    
    
    Type_next_to_nuclear_protein,Molecule_next_to_nuclear_protein,Type_next_to_promoter,Molecule_next_to_promoter,Type_next_to_enhancer,Type_next_to_enhancer=genenextto(Current_position_nuclear_protein,Current_position_enhancer,Current_position_promoter,nx,ny,Number_of_nuclear_protein,Number_enhancer,Number_promoter)
    
    #what can happened with single molecule and free site/occupied
    for i in range(Number_of_nuclear_protein):
        for j in range(4):
            #which TF can diffuse?
            if Type_next_to_nuclear_protein[i,j]==0 and Number_of_binding_of_nuclear_protein[i]==0:
                Time_to_diffuse_nuclear_protein[i,j]=np.random.exponential(scale=1/D_nuclear_protein, size=1)
            
            #TF binds to TF
            if Type_next_to_nuclear_protein[i,j]==1 and Molecule_bound_to_nuclear_protein[i,Molecule_next_to_nuclear_protein[i,j]]==0:
                ratelink=k_bind_nuclear_protein_to_nuclear_protein*(f-Number_of_binding_of_nuclear_protein[Molecule_next_to_nuclear_protein[i,j]])*(f-Number_of_binding_of_nuclear_protein[i])
                Time_to_link_nuclear_protein_to_nuclear_protein[i,Molecule_next_to_nuclear_protein[i,j]]=np.random.exponential(scale=1/ratelink,size=None)
                Time_to_link_nuclear_protein_to_nuclear_protein[Molecule_next_to_nuclear_protein[i,j],i]=Time_to_link_nuclear_protein_to_nuclear_protein[i,Molecule_next_to_nuclear_protein[i,j]]
            #TF unbinds TF
            if Type_next_to_nuclear_protein[i,j]==1 and Molecule_bound_to_nuclear_protein[i,Molecule_next_to_nuclear_protein[i,j]]==1:
                rateunb=k_unbinding_nuclear_protein_nuclear_protein/(Number_of_binding_of_nuclear_protein[Molecule_next_to_nuclear_protein[i,j]]*Number_of_binding_of_nuclear_protein[i])
                Time_to_unbind_nuclear_protein_to_nuclear_protein[i,Molecule_next_to_nuclear_protein[i,j]]=np.random.exponential(scale=1/rateunb,size=None)
                Time_to_unbind_nuclear_protein_to_nuclear_protein[Molecule_next_to_nuclear_protein[i,j],i]=Time_to_unbind_nuclear_protein_to_nuclear_protein[i,Molecule_next_to_nuclear_protein[i,j]]
                
             #TF binds to Enhancer
            if Type_next_to_nuclear_protein[i,j]==3 and Molecule_next_to_enhancer[i,Molecule_next_to_nuclear_protein[i,j]]==0:
                ratelink=k_link_nuclear_protein_enhancer*(f-Number_of_binding_of_enhancer[Molecule_next_to_nuclear_protein[i,j]])*(f-Number_of_binding_of_nuclear_protein[i])
                Time_to_link_nuclear_protein_to_enhancer[i,Molecule_next_to_nuclear_protein[i,j]]=np.random.exponential(scale=1/ratelink,size=None)    
            #TF unbinds Enhancer
            if Type_next_to_nuclear_protein[i,j]==3 and Molecule_next_to_enhancer[i,Molecule_next_to_nuclear_protein[i,j]]==1:
                rateunb=k_unbinding_nuclear_protein_enhancer/(Number_of_binding_of_enhancer[Molecule_next_to_nuclear_protein[i,j]]*Number_of_binding_of_nuclear_protein[i])
                Time_to_unbind_nuclear_protein_to_enhancer[i,Molecule_next_to_nuclear_protein[i,j]]=np.random.exponential(scale=1/rateunb,size=None)
            
            #TF binds to Prom
            if Type_next_to_nuclear_protein[i,j]==4 and Molecule_bound_to_prom[i,Molecule_next_to_nuclear_protein[i,j]]==0:
                ratelink=k_link_nuclear_protein_promoter**(f-Number_of_binding_of_promoter[Molecule_next_to_nuclear_protein[i,j]])*(f-Number_of_binding_of_nuclear_protein[i])
                Time_to_link_nuclear_protein_to_promoter[i,Molecule_next_to_nuclear_protein[i,j]]=np.random.exponential(scale=1/ratelink,size=None)
            #TF unbinds Prom
            if Type_next_to_nuclear_protein[i,j]==4 and Molecule_bound_to_prom[i,Molecule_next_to_nuclear_protein[i,j]]==1:
                rateunb=k_unbinding_nuclear_protein_promoter/(Number_of_binding_of_promoter[Molecule_next_to_nuclear_protein[i,j]]*Number_of_binding_of_nuclear_protein[i])
                Time_to_unbind_nuclear_protein_to_promoter[i,Molecule_next_to_nuclear_protein[i,j]]=np.random.exponential(scale=1/rateunb,size=None)
            
    #Promoter can only diffuse as proba of binding to another transcription factor is already determined            
    for i in range(Number_promoter):
        for j in range(4):
            #which Promoter can diffuse?
            if Type_next_to_promoter[i,j]==0 and Number_of_binding_of_promoter[i]==0:
                Time_to_diffuse_promoter[i,j]=np.random.exponential(scale=1/D_promoter, size=1)
    for i in range(Number_enhancer):
        for j in range(4):
            #which Polymerase can diffuse?
            if Type_next_to_enhancer[i,j]==0 and Number_of_binding_of_enhancer[i]==0:
                Time_to_diffuse_enhancer[i,j]=np.random.exponential(scale=1/D_enhancer, size=1)
    #which aggregate can move and in which direction
    for j in range(Number_of_aggregate):
        for h in range(4):
            movement=0
            chainTF=Nuclear_protein_in_aggregate[j][:Number_of_nuclear_protein_in_aggregate[j]]
            chainProm=Promoter_in_aggregate[j][:Number_of_promoter_in_aggregate[j]]
            chainEnh=Enhancer_in_aggregate[j][:Number_of_enhancer_in_aggregate[j]]
            #Can all TF move?
            for i in range(Number_of_nuclear_protein_in_aggregate[j]):
                #we will build a wall
                if Type_next_to_nuclear_protein[chainTF[i]][h]==2:
                    movement=1
                #TF does belong so it's ok
                if Type_next_to_nuclear_protein[chainTF[i],h]==1:
                        if (Molecule_next_to_nuclear_protein[chainTF[i],h]*np.ones(Number_of_nuclear_protein_in_aggregate[j])==chainTF).any():
                            nothing=42
                        else:
                            movement=1
                #Prom does belong so it's ok
                if Type_next_to_nuclear_protein[chainTF[i],h]==3:
                        if (Molecule_next_to_nuclear_protein[chainTF[i],h]*np.ones(Number_of_promoter_in_aggregate[j])==chainProm).any():
                            nothing=42
                        else:
                            movement=1
                #Pol does belong so it's ok
                if Type_next_to_nuclear_protein[chainTF[i],h]==4:
                        if (Molecule_next_to_nuclear_protein[chainTF[i],h]*np.ones(Number_of_enhancer_in_aggregate[j])==chainEnh).any():
                            nothing=42
                        else:
                            movement=1
            #Can all Promoter move?
            for i in range(Number_of_promoter_in_aggregate[j]):
                #is there a wall 
                if Type_next_to_promoter[chainProm[i]][h]==2:
                    movement=1
                #Tf does belong it's ok
                if Type_next_to_promoter[chainProm[i]][h]==1:
                        if (Molecule_next_to_promoter[chainProm[i],h]*np.ones(Number_of_nuclear_protein_in_aggregate[j])==chainTF).any():
                            nothing=42
                        else:
                            movement=1
                #Pol does belong so it's ok
                if Type_next_to_promoter[chainProm[i]][h]==3:
                        if (Molecule_next_to_promoter[chainProm[i],h]*np.ones(Number_of_promoter_in_aggregate[j])==chainProm).any():
                            nothing=42
                        else:
                            movement=1
                if Type_next_to_promoter[chainProm[i]][h]==4:
                        if (Molecule_next_to_promoter[chainProm[i],h]*np.ones(Number_of_enhancer_in_aggregate[j])==chainEnh).any():
                            nothing=42
                        else:
                            movement=1
            for i in range(Number_of_enhancer_in_aggregate[j]):
                #is there a wall 
                if Type_next_to_enhancer[chainEnh[i]][h]==2:
                    movement=1
                #Tf does belong it's ok
                if Type_next_to_enhancer[chainEnh[i]][h]==1:
                        if (Type_next_to_enhancer[chainEnh[i],h]*np.ones(Number_of_nuclear_protein_in_aggregate[j])==chainTF).any():
                            nothing=42
                        else:
                            movement=1
                #Prom does belong so it's ok
                if Type_next_to_enhancer[chainEnh[i]][h]==3:
                        if (Type_next_to_enhancer[chainEnh[i],h]*np.ones(Number_of_promoter_in_aggregate[j])==chainProm).any():
                            nothing=42
                        else:
                            movement=1
                if Type_next_to_enhancer[chainEnh[i]][h]==4:
                        if (Type_next_to_enhancer[chainEnh[i],h]*np.ones(Number_of_enhancer_in_aggregate[j])==chainEnh).any():
                            nothing=42
                        else:
                            movement=1               
            if movement==0:
                if Number_of_promoter_in_aggregate[j]==0 and Number_of_enhancer_in_aggregate[j]==0:
                    Deff=Number_of_nuclear_protein_in_aggregate[j]/D_nuclear_protein
                    Time_to_diffuse_aggregate[j,h]=np.random.exponential(1/Deff, size=None)
                if Number_of_promoter_in_aggregate[j]>0 and Number_of_enhancer_in_aggregate[j]==0:
                    Deff=pow((Number_of_nuclear_protein_in_aggregate[j]/D_nuclear_protein+Number_of_promoter_in_aggregate[j]/D_promoter),-1)
                    Time_to_diffuse_aggregate[j,h]=np.random.exponential(1/Deff, size=None)
                if Number_of_promoter_in_aggregate[j]>0 and Number_of_enhancer_in_aggregate[j]>0:
                    Deff=pow((Number_of_nuclear_protein_in_aggregate[j]/D_nuclear_protein+Number_of_promoter_in_aggregate[j]/D_promoter+Number_of_enhancer_in_aggregate[j]/D_enhancer),-1)
                    Time_to_diffuse_aggregate[j,h]=np.random.exponential(1/Deff, size=None)
                if Number_of_promoter_in_aggregate[j]==0 and Number_of_enhancer_in_aggregate[j]>0:
                    Deff=pow((Number_of_nuclear_protein_in_aggregate[j]/D_nuclear_protein+Number_of_enhancer_in_aggregate[j]/D_enhancer),-1)
                    Time_to_diffuse_aggregate[j,h]=np.random.exponential(1/Deff, size=None)                    
    for j in range(Number_of_aggregate):
        nblink=0
        for i in range(Number_of_nuclear_protein_in_aggregate[j]):
            nblink=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]
            
        if Number_of_PolII_in_aggregate[j]<PolII_per_free_link*nblink:
            ratepolbind=k_PolII_in_aggregate*pow(Number_of_nuclear_protein_in_aggregate[j],2/3)*(1-Number_of_PolII_in_aggregate[j]/(PolII_per_free_link*nblink))
            Time_to_bind_polII_to_aggregate[j]=np.random.exponential((1/ratepolbind),size=None)
        if Number_of_PolII_in_aggregate[j]>0:
            rateunbindpol=k_PolII_out_of_aggregate*pow(Number_of_nuclear_protein_in_aggregate[j],2/3)*Number_of_PolII_in_aggregate[j]
            Time_to_unbind_polII_from_aggregate[j]=np.random.exponential((1/rateunbindpol),size=None)
            
    Time_to_get_polII_on_promoter=1000*np.ones(Number_of_nuclear_protein)
    for i in range(Number_of_aggregate):
        if Number_of_enhancer_in_aggregate[j]>0 and Number_of_PolII_in_aggregate[j]>0:
            ratepolonProm=Number_of_PolII_in_aggregate[j]*Number_of_enhancer_in_aggregate[j]*k_PolII_on_promoter
            Time_to_get_polII_on_promoter[j]=np.random.exponential((1/ratepolonProm),size=None)
       
    if mRNA!=0:
        hi=0
        Time_to_degrade_mRNA=np.random.exponential((1/Delta_mRNA*mRNA), size=None)

    
    return Time_to_get_polII_on_promoter,Time_to_link_nuclear_protein_to_nuclear_protein,Time_to_link_nuclear_protein_to_enhancer,Time_to_link_nuclear_protein_to_promoter,Time_to_unbind_nuclear_protein_to_nuclear_protein,Time_to_unbind_nuclear_protein_to_enhancer,Time_to_unbind_nuclear_protein_to_promoter,Time_to_diffuse_enhancer,Time_to_diffuse_promoter,Time_to_diffuse_nuclear_protein,Time_to_degrade_mRNA,Time_to_diffuse_aggregate,Time_to_bind_polII_to_aggregate,Time_to_unbind_polII_from_aggregate

In [18]:
def TFDNAdif(PolII_per_free_link,k_PolII_on_promoter,k_PolII_in_aggregate,k_PolII_out_of_aggregate,Num_step,Current_position_enhancer,Current_position_promoter,Number_of_nuclear_protein,Number_promoter,Number_enhancer,Delta_mRNA,mesh,nx,ny,xmin,xmax,ymin,ymax,D_nuclear_protein,D_promoter,D_enhancer,f,k_bind_nuclear_protein_to_nuclear_protein,k_link_nuclear_protein_promoter,k_link_nuclear_protein_enhancer,k_unbinding_nuclear_protein_nuclear_protein,k_unbinding_nuclear_protein_promoter,k_unbinding_nuclear_protein_enhancer):
    
    #store the number of link for each type of molecules
    Number_of_binding_of_nuclear_protein=np.zeros(Number_of_nuclear_protein,int)
    Number_of_binding_of_promoter=np.zeros(Number_promoter,int)
    Number_of_binding_of_enhancer=np.zeros(Number_enhancer,int)
    
    #store which molecule is bound to which nuclear protein
    Molecule_bound_to_nuclear_protein=np.zeros((Number_of_nuclear_protein,Number_of_nuclear_protein),int)
    Molecule_bound_to_prom=np.zeros((Number_of_nuclear_protein,Number_promoter),int)
    Molecule_next_to_enhancer=np.zeros((Number_of_nuclear_protein,Number_enhancer),int)
    
    #store the molecules present in aggregate, one line representing on aggregate
    Nuclear_protein_in_aggregate=np.zeros((Number_of_nuclear_protein,Number_of_nuclear_protein),int)
    Promoter_in_aggregate=np.zeros((Number_of_nuclear_protein,Number_promoter),int)
    Enhancer_in_aggregate=np.zeros((Number_of_nuclear_protein,Number_enhancer),int)
    
    #store the number of each type of protein in aggregate, one line representing one aggregate
    Number_of_nuclear_protein_in_aggregate=np.zeros(Number_of_nuclear_protein,int)
    Number_of_promoter_in_aggregate=np.zeros(Number_of_nuclear_protein,int)
    Number_of_enhancer_in_aggregate=np.zeros(Number_of_nuclear_protein,int)
    Number_of_PolII_in_aggregate=np.zeros(Number_of_nuclear_protein,int)
    Number_of_aggregate=0
    
    #store trajectories for each type of molecules
    Trajectories_nuclear_protein=np.zeros((Num_step,Number_of_nuclear_protein),int)
    
    Trajectories_promoter=np.zeros((Num_step,Number_promoter),int)
    Trajectories_promoter[0,:]=Current_position_promoter
    
    Trajectories_enhancer=np.zeros((Num_step,Number_enhancer),int)
    Trajectories_enhancer[0,:]=Current_position_enhancer
    
    #initiate nuclear protein position
    Trajectories_nuclear_protein[0,:]=initTF(Current_position_promoter,Current_position_enhancer,Number_promoter,Number_enhancer,Number_of_nuclear_protein,mesh,nx,ny,xmin,xmax,ymin,ymax)
    Current_position_nuclear_protein=np.array(Trajectories_nuclear_protein[0,:],int)
    
    Time_vector=np.zeros(Num_step)
    
    Which_aggregate_contains_nuclear_protein=-1*np.ones(Number_of_nuclear_protein,int)
    Which_aggregate_contains_promoter=-1*np.ones(Number_promoter,int)
    Which_aggregate_contains_enhancer=-1*np.ones(Number_enhancer,int)
    
    #gene related
    Number_of_enhancer_in_aggregate_simulation=np.zeros((Num_step,Number_of_nuclear_protein),int)
    Number_of_promoter_in_aggregate_simulation=np.zeros((Num_step,Number_of_nuclear_protein),int)
    Number_of_nuclear_protein_in_aggregate_simulation=np.zeros((Num_step,Number_of_nuclear_protein),int)
    Number_of_PolII_in_aggregate_simulation=np.zeros((Num_step,Number_of_nuclear_protein),int)
    
    mRNA=0
    mRNAvec=np.zeros(Num_step,int)
    
    for T_index in range(Num_step):
        #generation of probability matrix
        
        Time_to_get_polII_on_promoter,Time_to_link_nuclear_protein_to_nuclear_protein,Time_to_link_nuclear_protein_to_enhancer,Time_to_link_nuclear_protein_to_promoter,Time_to_unbind_nuclear_protein_to_nuclear_protein,Time_to_unbind_nuclear_protein_to_enhancer,Time_to_unbind_nuclear_protein_to_promoter,Time_to_diffuse_enhancer,Time_to_diffuse_promoter,Time_to_diffuse_nuclear_protein,Time_to_degrade_mRNA,Time_to_diffuse_aggregate,Time_to_bind_polII_to_aggregate,Time_to_unbind_polII_from_aggregate=probagenerator(PolII_per_free_link,k_PolII_on_promoter,k_PolII_in_aggregate,k_PolII_out_of_aggregate,Number_of_PolII_in_aggregate,Number_promoter,Number_enhancer,k_link_nuclear_protein_promoter,k_link_nuclear_protein_enhancer,k_unbinding_nuclear_protein_promoter,k_unbinding_nuclear_protein_enhancer,D_enhancer,D_promoter,Number_of_nuclear_protein,Delta_mRNA,mRNA,Current_position_nuclear_protein,Current_position_promoter,Current_position_enhancer,nx,ny,Number_of_binding_of_nuclear_protein,D_nuclear_protein,Molecule_bound_to_nuclear_protein,k_bind_nuclear_protein_to_nuclear_protein,f,Molecule_bound_to_prom,Number_of_binding_of_promoter,Molecule_next_to_enhancer,Number_of_binding_of_enhancer,k_unbinding_nuclear_protein_nuclear_protein,Number_of_aggregate,Nuclear_protein_in_aggregate,Promoter_in_aggregate,Enhancer_in_aggregate,Number_of_nuclear_protein_in_aggregate,Number_of_promoter_in_aggregate,Number_of_enhancer_in_aggregate)
        
        
        #now we compare time
        Min_time_to_link_nuclear_protein_to_nuclear_protein=Time_to_link_nuclear_protein_to_nuclear_protein.min()
        Min_Time_to_link_nuclear_protein_to_enhancer=Time_to_link_nuclear_protein_to_promoter.min()
        Min_Time_to_link_nuclear_protein_to_promoter=Time_to_link_nuclear_protein_to_enhancer.min()
        
        Min_Time_to_unbind_nuclear_protein_to_nuclear_protein=Time_to_unbind_nuclear_protein_to_nuclear_protein.min()
        Min_Time_to_unbind_nuclear_protein_to_promoter=Time_to_unbind_nuclear_protein_to_promoter.min()
        Min_Time_to_unbind_nuclear_protein_to_enhancer=Time_to_unbind_nuclear_protein_to_enhancer.min()
        
        Min_Time_to_diffuse_nuclear_protein=Time_to_diffuse_nuclear_protein.min()
        Min_Time_to_diffuse_promoter=Time_to_diffuse_promoter.min()
        Min_Time_to_diffuse_enhancer=Time_to_diffuse_enhancer.min()
        Min_Time_to_diffuse_aggregate=Time_to_diffuse_aggregate.min()
        
        Min_Time_to_bind_polII_to_aggregate=Time_to_bind_polII_to_aggregate.min()
        Min_Time_to_unbind_polII_from_aggregate=Time_to_unbind_polII_from_aggregate.min()
        Min_Time_to_get_polII_on_promoter=Time_to_get_polII_on_promoter.min()
        
        t=min(Time_to_degrade_mRNA,Min_Time_to_get_polII_on_promoter,Min_Time_to_bind_polII_to_aggregate,Min_Time_to_unbind_polII_from_aggregate,Min_time_to_link_nuclear_protein_to_nuclear_protein,Min_Time_to_link_nuclear_protein_to_enhancer,Min_Time_to_unbind_nuclear_protein_to_nuclear_protein,Min_Time_to_link_nuclear_protein_to_promoter,Min_Time_to_unbind_nuclear_protein_to_promoter,Min_Time_to_unbind_nuclear_protein_to_enhancer,Min_Time_to_diffuse_nuclear_protein,Min_Time_to_diffuse_enhancer,Min_Time_to_diffuse_promoter,Min_Time_to_diffuse_aggregate)
        
        
        #Pol linked
        if t==Min_Time_to_get_polII_on_promoter:
            index=np.argmin(Time_to_get_polII_on_promoter)
            Number_of_PolII_in_aggregate[index]=Number_of_PolII_in_aggregate[index]-1
            mRNA=mRNA+1
            
        if t==Time_to_degrade_mRNA:
            mRNA=mRNA-1
        
        if t==Min_Time_to_bind_polII_to_aggregate:
            index=np.argmin(Time_to_bind_polII_to_aggregate)
            Number_of_PolII_in_aggregate[index]=Number_of_PolII_in_aggregate[index]+1
            
        if t==Min_Time_to_unbind_polII_from_aggregate:
            index=np.argmin(Time_to_unbind_polII_from_aggregate)
            Number_of_PolII_in_aggregate[index]=Number_of_PolII_in_aggregate[index]-1
        
        #diffusion linked
        
        if t==Min_Time_to_diffuse_nuclear_protein:
            index=np.argmin(Time_to_diffuse_nuclear_protein)
            Molecule_moving=index//4
            Direction_move=index%4
            if Direction_move==0 :
                Current_position_nuclear_protein[Molecule_moving]=Current_position_nuclear_protein[Molecule_moving]+nx
            elif Direction_move==1 :
                Current_position_nuclear_protein[Molecule_moving]=Current_position_nuclear_protein[Molecule_moving]+1
            elif Direction_move==2 :
                Current_position_nuclear_protein[Molecule_moving]=Current_position_nuclear_protein[Molecule_moving]-nx
            elif Direction_move==3 :
                Current_position_nuclear_protein[Molecule_moving]=Current_position_nuclear_protein[Molecule_moving]-1
        #diffusion of Prom
        elif t==Min_Time_to_diffuse_promoter:
            index=np.argmin(Time_to_diffuse_promoter)
            Molecule_moving=index//4
            Direction_move=index%4
            if Direction_move==0 :
                Current_position_promoter[Molecule_moving]=Current_position_promoter[Molecule_moving]+nx
            elif Direction_move==1 :
                Current_position_promoter[Molecule_moving]=Current_position_promoter[Molecule_moving]+1
            elif Direction_move==2 :
                Current_position_promoter[Molecule_moving]=Current_position_promoter[Molecule_moving]-nx
            elif Direction_move==3 :
                Current_position_promoter[Molecule_moving]=Current_position_promoter[Molecule_moving]-1
        
        #diffusion of Enh
        elif t==Min_Time_to_diffuse_enhancer:
            index=np.argmin(Time_to_diffuse_enhancer)
            Molecule_moving=index//4
            Direction_move=index%4
            if Direction_move==0 :
                Current_position_enhancer[Molecule_moving]=Current_position_enhancer[Molecule_moving]+nx
            elif Direction_move==1 :
                Current_position_enhancer[Molecule_moving]=Current_position_enhancer[Molecule_moving]+1
            elif Direction_move==2 :
                Current_position_enhancer[Molecule_moving]=Current_position_enhancer[Molecule_moving]-nx
            elif Direction_move==3 :
                Current_position_enhancer[Molecule_moving]=Current_position_enhancer[Molecule_moving]-1
        
        #diffusion of aggregate
        if t==Min_Time_to_diffuse_aggregate:
            index=np.argmin(Time_to_diffuse_aggregate)
            aggmove=index//4
            Direction_move=index%4
            for nbaggTF in range(Number_of_nuclear_protein_in_aggregate[aggmove]):
                if Direction_move==0:
                    Current_position_nuclear_protein[Nuclear_protein_in_aggregate[aggmove][nbaggTF]]=Current_position_nuclear_protein[Nuclear_protein_in_aggregate[aggmove][nbaggTF]]+nx
                elif Direction_move==1:
                    Current_position_nuclear_protein[Nuclear_protein_in_aggregate[aggmove][nbaggTF]]=Current_position_nuclear_protein[Nuclear_protein_in_aggregate[aggmove][nbaggTF]]+1
                elif Direction_move==2:
                    Current_position_nuclear_protein[Nuclear_protein_in_aggregate[aggmove][nbaggTF]]=Current_position_nuclear_protein[Nuclear_protein_in_aggregate[aggmove][nbaggTF]]-nx
                elif Direction_move==3:
                    Current_position_nuclear_protein[Nuclear_protein_in_aggregate[aggmove][nbaggTF]]=Current_position_nuclear_protein[Nuclear_protein_in_aggregate[aggmove][nbaggTF]]-1
            for nbaggProm in range(Number_of_promoter_in_aggregate[aggmove]):
                if Direction_move==0:
                    Current_position_promoter[Promoter_in_aggregate[aggmove][nbaggProm]]=Current_position_promoter[Promoter_in_aggregate[aggmove][nbaggProm]]+nx
                elif Direction_move==1:
                    Current_position_promoter[Promoter_in_aggregate[aggmove][nbaggProm]]=Current_position_promoter[Promoter_in_aggregate[aggmove][nbaggProm]]+1
                elif Direction_move==2:
                    Current_position_promoter[Promoter_in_aggregate[aggmove][nbaggProm]]=Current_position_promoter[Promoter_in_aggregate[aggmove][nbaggProm]]-nx
                elif Direction_move==3:
                    Current_position_promoter[Promoter_in_aggregate[aggmove][nbaggProm]]=Current_position_promoter[Promoter_in_aggregate[aggmove][nbaggProm]]-1
              
            for nbaggEnh in range(Number_of_enhancer_in_aggregate[aggmove]):
                if Direction_move==0:
                    Current_position_enhancer[Enhancer_in_aggregate[aggmove][nbaggEnh]]=Current_position_enhancer[Enhancer_in_aggregate[aggmove][nbaggEnh]]+nx
                elif Direction_move==1:
                    Current_position_enhancer[Enhancer_in_aggregate[aggmove][nbaggEnh]]=Current_position_enhancer[Enhancer_in_aggregate[aggmove][nbaggEnh]]+1
                elif Direction_move==2:
                    Current_position_enhancer[Enhancer_in_aggregate[aggmove][nbaggEnh]]=Current_position_enhancer[Enhancer_in_aggregate[aggmove][nbaggEnh]]-nx
                elif Direction_move==3:
                    Current_position_enhancer[Enhancer_in_aggregate[aggmove][nbaggEnh]]=Current_position_enhancer[Enhancer_in_aggregate[aggmove][nbaggEnh]]-1
             
        
        #binding b/w TF and TF
        elif t==Min_time_to_link_nuclear_protein_to_nuclear_protein:
            index=np.argmin(Time_to_link_nuclear_protein_to_nuclear_protein)
            mol1=index//Number_of_nuclear_protein
            mol2=index%Number_of_nuclear_protein
            
            Number_of_binding_of_nuclear_protein[mol1]=Number_of_binding_of_nuclear_protein[mol1]+1
            Number_of_binding_of_nuclear_protein[mol2]=Number_of_binding_of_nuclear_protein[mol2]+1
            Molecule_bound_to_nuclear_protein[mol1,mol2]=1
            Molecule_bound_to_nuclear_protein[mol2,mol1]=1
            if Which_aggregate_contains_nuclear_protein[mol1]==-1 and Which_aggregate_contains_nuclear_protein[mol2]==-1:
               
                Nuclear_protein_in_aggregate[Number_of_aggregate][0]=mol1
                Nuclear_protein_in_aggregate[Number_of_aggregate][1]=mol2
                Number_of_nuclear_protein_in_aggregate[Number_of_aggregate]=2
                Which_aggregate_contains_nuclear_protein[mol1]=Number_of_aggregate
                Which_aggregate_contains_nuclear_protein[mol2]=Number_of_aggregate
                Number_of_aggregate=Number_of_aggregate+1
                
            elif Which_aggregate_contains_nuclear_protein[mol1]!=-1 and Which_aggregate_contains_nuclear_protein[mol2]==-1:
                Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]]=mol2
                Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+1
                Which_aggregate_contains_nuclear_protein[mol2]=Which_aggregate_contains_nuclear_protein[mol1]
            elif Which_aggregate_contains_nuclear_protein[mol1]==-1 and Which_aggregate_contains_nuclear_protein[mol2]!=-1:
                Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]]=mol1
                Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]+1
                Which_aggregate_contains_nuclear_protein[mol1]=Which_aggregate_contains_nuclear_protein[mol2]
            elif Which_aggregate_contains_nuclear_protein[mol1]!=-1 and Which_aggregate_contains_nuclear_protein[mol2]!=-1:
                if Which_aggregate_contains_nuclear_protein[mol1]!=Which_aggregate_contains_nuclear_protein[mol2]:
                    
                    remagg=Which_aggregate_contains_nuclear_protein[mol2]
                    #first we put the TF in the right place
                    for i in range(Number_of_nuclear_protein_in_aggregate[remagg]):
                        Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Nuclear_protein_in_aggregate[remagg][i]
                        Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_nuclear_protein_in_aggregate[remagg]
                    
                    #then we put the prom in the right place
                    for i in range(Number_of_promoter_in_aggregate[remagg]):
                        Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Promoter_in_aggregate[remagg][i]
                        Which_aggregate_contains_promoter[Promoter_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_promoter_in_aggregate[remagg]
                    
                    #then we put the enh in the right place
                    for i in range(Number_of_enhancer_in_aggregate[remagg]):
                        Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Enhancer_in_aggregate[remagg][i]
                        Which_aggregate_contains_enhancer[Enhancer_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_enhancer_in_aggregate[remagg]
                    
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_PolII_in_aggregate[remagg]
                    
                    #we replace chains in TF
                    for j in range(remagg,Number_of_aggregate-1):
                        Nuclear_protein_in_aggregate[j][:]=Nuclear_protein_in_aggregate[j+1][:]
                        for i in range(Number_of_nuclear_protein_in_aggregate[j+1]):
                            Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]=Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]-1
                        Number_of_nuclear_protein_in_aggregate[j]=Number_of_nuclear_protein_in_aggregate[j+1]
                        Number_of_PolII_in_aggregate[j]=Number_of_PolII_in_aggregate[j+1]
                    Nuclear_protein_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_of_nuclear_protein)
                    Number_of_nuclear_protein_in_aggregate[Number_of_aggregate-1]=0
                    Number_of_PolII_in_aggregate[Number_of_aggregate-1]=0
                    
                    #we replace chains in Prom
                    for j in range(remagg,Number_of_aggregate-1):
                        Promoter_in_aggregate[j][:]=Promoter_in_aggregate[j+1][:]
                        for i in range(Number_of_promoter_in_aggregate[j+1]):
                            Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]=Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]-1
                        Number_of_promoter_in_aggregate[j]=Number_of_promoter_in_aggregate[j+1]
                    Promoter_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_promoter)
                    Number_of_promoter_in_aggregate[Number_of_aggregate-1]=0
                    
                    #we replace chains in Enh
                    for j in range(remagg,Number_of_aggregate-1):
                        Enhancer_in_aggregate[j][:]=Enhancer_in_aggregate[j+1][:]
                        for i in range(Number_of_enhancer_in_aggregate[j+1]):
                            Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]=Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]-1
                        Number_of_enhancer_in_aggregate[j]=Number_of_enhancer_in_aggregate[j+1]
                    Enhancer_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_enhancer)
                    Number_of_enhancer_in_aggregate[Number_of_aggregate-1]=0
                    
                    Number_of_aggregate=Number_of_aggregate-1
                    
        elif t==Min_Time_to_link_nuclear_protein_to_enhancer:
            index=np.argmin(Time_to_link_nuclear_protein_to_promoter)
            mol1=index//Number_promoter
            mol2=index%Number_promoter
            
            Number_of_binding_of_nuclear_protein[mol1]=Number_of_binding_of_nuclear_protein[mol1]+1
            Number_of_binding_of_promoter[mol2]=Number_of_binding_of_promoter[mol2]+1
            Molecule_bound_to_prom[mol1,mol2]=1
            if Which_aggregate_contains_nuclear_protein[mol1]==-1 and Which_aggregate_contains_promoter[mol2]==-1:
                Nuclear_protein_in_aggregate[Number_of_aggregate][0]=mol1
                Promoter_in_aggregate[Number_of_aggregate][0]=mol2
                Number_of_nuclear_protein_in_aggregate[Number_of_aggregate]=1
                Number_of_promoter_in_aggregate[Number_of_aggregate]=1
                Which_aggregate_contains_nuclear_protein[mol1]=Number_of_aggregate
                Which_aggregate_contains_promoter[mol2]=Number_of_aggregate
                Number_of_aggregate=Number_of_aggregate+1
                
            elif Which_aggregate_contains_nuclear_protein[mol1]!=-1 and Which_aggregate_contains_promoter[mol2]==-1:
                Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]]=mol2
                Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+1
                Which_aggregate_contains_promoter[mol2]=Which_aggregate_contains_nuclear_protein[mol1]
            elif Which_aggregate_contains_nuclear_protein[mol1]==-1 and Which_aggregate_contains_promoter[mol2]!=-1:
                
                Nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]]]=mol1
                Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]]+1
                Which_aggregate_contains_nuclear_protein[mol1]=Which_aggregate_contains_promoter[mol2]
            elif Which_aggregate_contains_nuclear_protein[mol1]!=-1 and Which_aggregate_contains_promoter[mol2]!=-1:
                if Which_aggregate_contains_nuclear_protein[mol1]!=Which_aggregate_contains_promoter[mol2]:
                    
                    remagg=Which_aggregate_contains_promoter[mol2]
                    #first we put the TF in the right place
                    for i in range(Number_of_nuclear_protein_in_aggregate[remagg]):
                        Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Nuclear_protein_in_aggregate[remagg][i]
                        Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_nuclear_protein_in_aggregate[remagg]
                    
                    #then we put the prom in the right place
                    for i in range(Number_of_promoter_in_aggregate[remagg]):
                        Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Promoter_in_aggregate[remagg][i]
                        Which_aggregate_contains_promoter[Promoter_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_promoter_in_aggregate[remagg]
                    
                    #then we put the Enh in the right place
                    for i in range(Number_of_enhancer_in_aggregate[remagg]):
                        Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Enhancer_in_aggregate[remagg][i]
                        Which_aggregate_contains_enhancer[Enhancer_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_enhancer_in_aggregate[remagg]
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_PolII_in_aggregate[remagg]
                    Number_of_PolII_in_aggregate[Number_of_aggregate-1]=0
                    
                    #we replace chains in TF
                    for j in range(remagg,Number_of_aggregate-1):
                        Nuclear_protein_in_aggregate[j][:]=Nuclear_protein_in_aggregate[j+1][:]
                        for i in range(Number_of_nuclear_protein_in_aggregate[j+1]):
                            Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]=Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]-1
                        Number_of_nuclear_protein_in_aggregate[j]=Number_of_nuclear_protein_in_aggregate[j+1]
                        Number_of_PolII_in_aggregate[j]=Number_of_PolII_in_aggregate[j+1]
                    
                    Nuclear_protein_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_of_nuclear_protein)
                    Number_of_nuclear_protein_in_aggregate[Number_of_aggregate-1]=0
                    
                    Number_of_PolII_in_aggregate[Number_of_aggregate-1]=0
                    #we replace chains in Prom
                    for j in range(remagg,Number_of_aggregate-1):
                        Promoter_in_aggregate[j][:]=Promoter_in_aggregate[j+1][:]
                        for i in range(Number_of_promoter_in_aggregate[j+1]):
                            Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]=Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]-1
                        Number_of_promoter_in_aggregate[j]=Number_of_promoter_in_aggregate[j+1]
                    Promoter_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_promoter)
                    Number_of_promoter_in_aggregate[Number_of_aggregate-1]=0
                    
                    #we replace chains in Enh
                    for j in range(remagg,Number_of_aggregate-1):
                        Enhancer_in_aggregate[j][:]=Enhancer_in_aggregate[j+1][:]
                        for i in range(Number_of_enhancer_in_aggregate[j+1]):
                            Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]=Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]-1
                        Number_of_enhancer_in_aggregate[j]=Number_of_enhancer_in_aggregate[j+1]
                    Enhancer_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_enhancer)
                    Number_of_enhancer_in_aggregate[Number_of_aggregate-1]=0
                    
                    Number_of_aggregate=Number_of_aggregate-1
        
        # we link TF and enhancer
        elif t==Min_Time_to_link_nuclear_protein_to_promoter:
            index=np.argmin(Time_to_link_nuclear_protein_to_enhancer)
            mol1=index//Number_enhancer
            mol2=index%Number_enhancer
            
            Number_of_binding_of_nuclear_protein[mol1]=Number_of_binding_of_nuclear_protein[mol1]+1
            Number_of_binding_of_enhancer[mol2]=Number_of_binding_of_enhancer[mol2]+1
            Molecule_next_to_enhancer[mol1,mol2]=1
            if Which_aggregate_contains_nuclear_protein[mol1]==-1 and Which_aggregate_contains_enhancer[mol2]==-1:
                Nuclear_protein_in_aggregate[Number_of_aggregate][0]=mol1
                Enhancer_in_aggregate[Number_of_aggregate][0]=mol2
                Number_of_nuclear_protein_in_aggregate[Number_of_aggregate]=1
                Number_of_enhancer_in_aggregate[Number_of_aggregate]=1
                Which_aggregate_contains_nuclear_protein[mol1]=Number_of_aggregate
                Which_aggregate_contains_enhancer[mol2]=Number_of_aggregate
                Number_of_aggregate=Number_of_aggregate+1
                
            elif Which_aggregate_contains_nuclear_protein[mol1]!=-1 and Which_aggregate_contains_enhancer[mol2]==-1:
                Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]]=mol2
                Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+1
                Which_aggregate_contains_enhancer[mol2]=Which_aggregate_contains_nuclear_protein[mol1]
            elif Which_aggregate_contains_nuclear_protein[mol1]==-1 and Which_aggregate_contains_enhancer[mol2]!=-1:
                Nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]]]=mol1
                Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]]+1
                Which_aggregate_contains_nuclear_protein[mol1]=Which_aggregate_contains_enhancer[mol2]
            elif Which_aggregate_contains_nuclear_protein[mol1]!=-1 and Which_aggregate_contains_enhancer[mol2]!=-1:
                if Which_aggregate_contains_nuclear_protein[mol1]!=Which_aggregate_contains_enhancer[mol2]:
                    remagg=Which_aggregate_contains_enhancer[mol2]
                    #first we put the TF in the right place
                    for i in range(Number_of_nuclear_protein_in_aggregate[remagg]):
                        Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Nuclear_protein_in_aggregate[remagg][i]
                        Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_nuclear_protein_in_aggregate[remagg]
                    
                    #then we put the prom in the right place
                    for i in range(Number_of_promoter_in_aggregate[remagg]):
                        Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Promoter_in_aggregate[remagg][i]
                        Which_aggregate_contains_promoter[Promoter_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_promoter_in_aggregate[remagg]
                    
                    #then we put the Enh in the right place
                    for i in range(Number_of_enhancer_in_aggregate[remagg]):
                        Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+i]=Enhancer_in_aggregate[remagg][i]
                        Which_aggregate_contains_enhancer[Enhancer_in_aggregate[remagg][i]]=Which_aggregate_contains_nuclear_protein[mol1]
                    Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_enhancer_in_aggregate[remagg]
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]+Number_of_PolII_in_aggregate[remagg]
                   
                    
                    
                    #we replace chains in TF
                    for j in range(remagg,Number_of_aggregate-1):
                        Nuclear_protein_in_aggregate[j][:]=Nuclear_protein_in_aggregate[j+1][:]
                        for i in range(Number_of_nuclear_protein_in_aggregate[j+1]):
                            Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]=Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]-1
                        Number_of_nuclear_protein_in_aggregate[j]=Number_of_nuclear_protein_in_aggregate[j+1]
                        Number_of_PolII_in_aggregate[j]=Number_of_PolII_in_aggregate[j+1]
                    Nuclear_protein_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_of_nuclear_protein)
                    Number_of_nuclear_protein_in_aggregate[Number_of_aggregate-1]=0
                    Number_of_PolII_in_aggregate[Number_of_aggregate-1]=0
                    #we replace chains in Prom
                    for j in range(remagg,Number_of_aggregate-1):
                        Promoter_in_aggregate[j][:]=Promoter_in_aggregate[j+1][:]
                        for i in range(Number_of_promoter_in_aggregate[j+1]):
                            Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]=Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]-1
                        Number_of_promoter_in_aggregate[j]=Number_of_promoter_in_aggregate[j+1]
                    Promoter_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_promoter)
                    Number_of_promoter_in_aggregate[Number_of_aggregate-1]=0
                    
                    #we replace chains in Enh
                    for j in range(remagg,Number_of_aggregate-1):
                        Enhancer_in_aggregate[j][:]=Enhancer_in_aggregate[j+1][:]
                        for i in range(Number_of_enhancer_in_aggregate[j+1]):
                            Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]=Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]-1
                        Number_of_enhancer_in_aggregate[j]=Number_of_enhancer_in_aggregate[j+1]
                    Enhancer_in_aggregate[Number_of_aggregate-1][:]=np.zeros(Number_enhancer)
                    Number_of_enhancer_in_aggregate[Number_of_aggregate-1]=0
                    
                    Number_of_aggregate=Number_of_aggregate-1
                                
        #unbind of two TF            
        elif t==Min_Time_to_unbind_nuclear_protein_to_nuclear_protein:
            index=np.argmin(Time_to_unbind_nuclear_protein_to_nuclear_protein)
            mol1=index//Number_of_nuclear_protein
            mol2=index%Number_of_nuclear_protein
            
            Number_of_binding_of_nuclear_protein[mol1]=Number_of_binding_of_nuclear_protein[mol1]-1
            Number_of_binding_of_nuclear_protein[mol2]=Number_of_binding_of_nuclear_protein[mol2]-1
            Molecule_bound_to_nuclear_protein[mol1,mol2]=0
            Molecule_bound_to_nuclear_protein[mol2,mol1]=0
            
            if (Number_of_binding_of_nuclear_protein[mol1]==0 and Number_of_binding_of_nuclear_protein[mol2]==0):
                Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=0
                for j in range(Which_aggregate_contains_nuclear_protein[mol1],Number_of_aggregate):
                    Nuclear_protein_in_aggregate[j][:]=Nuclear_protein_in_aggregate[j+1][:]
                    for i in range(Number_of_nuclear_protein_in_aggregate[j+1]):
                            Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]=Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]-1
                    Number_of_nuclear_protein_in_aggregate[j]=Number_of_nuclear_protein_in_aggregate[j+1]
                    
                    Promoter_in_aggregate[j][:]=Promoter_in_aggregate[j+1][:]
                    for i in range(Number_of_promoter_in_aggregate[j+1]):
                            Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]=Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]-1
                    Number_of_promoter_in_aggregate[j]=Number_of_promoter_in_aggregate[j+1]        
                            
                    Enhancer_in_aggregate[j][:]=Enhancer_in_aggregate[j+1][:]        
                    for i in range(Number_of_enhancer_in_aggregate[j+1]):
                            Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]=Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]-1
                    Number_of_enhancer_in_aggregate[j]=Number_of_enhancer_in_aggregate[j+1]
                    
                Which_aggregate_contains_nuclear_protein[mol1]=-1
                Which_aggregate_contains_nuclear_protein[mol2]=-1
                Number_of_aggregate=Number_of_aggregate-1
            elif (Number_of_binding_of_nuclear_protein[mol1]==0 and Number_of_binding_of_nuclear_protein[mol2]!=0):
                bg=0
                while Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][bg]!=mol1:
                    bg=bg+1
                for j in range(bg,Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1):
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j]=Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j+1]
                Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1]=0
                Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1
                Which_aggregate_contains_nuclear_protein[mol1]=-1
                
                nblink=0
                for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]):
                    nblink=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]][i]]
                retnbpol=0
                for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]):
                    test=np.random.uniform(low=0,high=1,size=None)
                    if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]-nblink-1+3)):
                        retnbpol=retnbpol+1
                Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]=retnbpol
                
            elif (Number_of_binding_of_nuclear_protein[mol1]!=0 and Number_of_binding_of_nuclear_protein[mol2]==0):
                bg=0
                while Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]][bg]!=mol2:
                        bg=bg+1
                for j in range(bg,Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]-1):
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]][j]=Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]][j+1]
                Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]-1]=0
                Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]-1
                Which_aggregate_contains_nuclear_protein[mol2]=-1
                
                nblink=0
                for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                    nblink=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][i]]
                retnbpol=0
                for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                    test=np.random.uniform(low=0,high=1,size=None)
                    if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1+3)):
                        retnbpol=retnbpol+1
                Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]=retnbpol
            #What if the two molecules dissociate also form 2 aggregate
            else:
                chain1TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                chain1Prom=-1*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                chain1Enh=-1*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                
                markchain1TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                markchain1TF[0]=mol1
                markTF=0
                
                sizechain1TF=1
                sizechain1Prom=0
                sizechain1Enh=0
                
                while ((markchain1TF[:]!=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any()):

                    for i in range(Number_of_nuclear_protein):
                        if Molecule_bound_to_nuclear_protein[markchain1TF[markTF],i]==1:
                            if (chain1TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                oposite=0
                            else:
                                markchain1TF[sizechain1TF]=i
                                sizechain1TF=sizechain1TF+1
                    #is the marked TF linked to a promoter
                    for j in range(Number_promoter):  
                        if Molecule_bound_to_prom[markchain1TF[markTF],j]==1:
                            if (chain1Prom==j*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                opposite=0
                            else:
                                chain1Prom[sizechain1Prom]=j
                                sizechain1Prom=sizechain1Prom+1
                                #are those promoter linked to another TF
                                for p in range(Number_of_nuclear_protein):
                                    if Molecule_bound_to_prom[p,j]==1:
                                        if (chain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                            oposite=0
                                        else:
                                            markchain1TF[sizechain1TF]=p
                                            sizechain1TF=sizechain1TF+1
                    for j in range(Number_enhancer):  
                        if Molecule_next_to_enhancer[markchain1TF[markTF],j]==1:
                            if (chain1Enh==j*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                opposite=0
                            else:
                                chain1Enh[sizechain1Enh]=j
                                sizechain1Enh=sizechain1Enh+1
                                #are those polymerase linked to another TF
                                for p in range(Number_of_nuclear_protein):
                                    if Molecule_next_to_enhancer[p,j]==1:
                                        if (chain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                            oposite=0
                                        else:
                                            markchain1TF[sizechain1TF]=p
                                            sizechain1TF=sizechain1TF+1
                                                                     
                    chain1TF[markTF]=markchain1TF[markTF]
                    markchain1TF[markTF]=-1
                    markTF=markTF+1
                    
                    
                if sizechain1TF!=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]:
                    #print('YES WE BREAK FCKING EVERYTHING')
                    chain2TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    chain2Prom=-1*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    chain2Enh=-1*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    
                    markchain2TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    markchain2TF[0]=mol2
                    
                    markTF=0
                    sizechain2TF=1
                    sizechain2Prom=0
                    sizechain2Enh=0
                    
                    while ((markchain2TF[:]!=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any()):

                        for i in range(Number_of_nuclear_protein):
                            if Molecule_bound_to_nuclear_protein[markchain2TF[markTF],i]==1:
                                if (chain2TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    markchain2TF[sizechain2TF]=i
                                    sizechain2TF=sizechain2TF+1

                        for j in range(Number_promoter):  
                            if Molecule_bound_to_prom[markchain2TF[markTF],j]==1:

                                if (chain2Prom==j*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    chain2Prom[sizechain2Prom]=j
                                    sizechain2Prom=sizechain2Prom+1
                                    for p in range(Number_of_nuclear_protein):
                                        if Molecule_bound_to_prom[p,j]==1:
                                            if (chain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                                oposite=0
                                            else:
                                                markchain2TF[sizechain2TF]=p
                                                sizechain2TF=sizechain2TF+1
                        
                        for j in range(Number_enhancer):  
                            if Molecule_next_to_enhancer[markchain2TF[markTF],j]==1:
                                if (chain2Enh==j*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    chain2Enh[sizechain2Enh]=j
                                    sizechain2Enh=sizechain2Enh+1
                                    
                                    for p in range(Number_of_nuclear_protein):
                                        if Molecule_next_to_enhancer[p,j]==1:
                                            if (chain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                                oposite=0
                                            else:
                                                markchain2TF[sizechain2TF]=p
                                                sizechain2TF=sizechain2TF+1
                                                
                        chain2TF[markTF]=markchain2TF[markTF]
                        markchain2TF[markTF]=-1
                        markTF=markTF+1
                 
                    Nuclear_protein_in_aggregate[Number_of_aggregate][:sizechain2TF]=chain2TF[:sizechain2TF]
                    Promoter_in_aggregate[Number_of_aggregate][:sizechain2Prom]=chain2Prom[:sizechain2Prom]
                    Enhancer_in_aggregate[Number_of_aggregate][:sizechain2Enh]=chain2Enh[:sizechain2Enh]
                    Number_of_nuclear_protein_in_aggregate[Number_of_aggregate]=sizechain2TF
                    Number_of_promoter_in_aggregate[Number_of_aggregate]=sizechain2Prom
                    Number_of_enhancer_in_aggregate[Number_of_aggregate]=sizechain2Enh
                    
                    for i in range(sizechain2TF):
                        Which_aggregate_contains_nuclear_protein[chain2TF[i]]=Number_of_aggregate
                    for i in range(sizechain2Prom):
                        Which_aggregate_contains_promoter[chain2Prom[i]]=Number_of_aggregate
                    for i in range(sizechain2Enh):
                        Which_aggregate_contains_enhancer[chain2Enh[i]]=Number_of_aggregate    
                    
                    Number_of_aggregate=Number_of_aggregate+1
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_of_nuclear_protein)
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1TF]=chain1TF[:sizechain1TF]
                    Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_promoter)
                    Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1Prom]=chain1Prom[:sizechain1Prom]
                    Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_enhancer)
                    Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1Enh]=chain1Enh[:sizechain1Enh]
                    
                    Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2TF
                    Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2Prom
                    Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2Enh
                    
                    Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]=sizechain2TF
                    Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]=sizechain2Prom
                    Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]=sizechain2Enh
                    
                    nblink=0
                    for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                        nblinkmol1=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][i]]
                    for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]):
                        nblinkmol2=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]][i]]
                        
                    retnbpol=0
                    for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                        test=np.random.uniform(low=0,high=1,size=None)
                        if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblinkmol1+4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]-nblinkmol2-2)):
                            retnbpol=retnbpol+1
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol2]]=Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-retnbpol
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=retnbpol
                    
        elif t==Min_Time_to_unbind_nuclear_protein_to_promoter:
                
            index=np.argmin(Time_to_unbind_nuclear_protein_to_promoter)
            mol1=index//Number_promoter
            mol2=index%Number_promoter
            
            
            Number_of_binding_of_nuclear_protein[mol1]=Number_of_binding_of_nuclear_protein[mol1]-1
            Number_of_binding_of_promoter[mol2]=Number_of_binding_of_promoter[mol2]-1
            Molecule_bound_to_prom[mol1,mol2]=0
            
            
            if (Number_of_binding_of_nuclear_protein[mol1]==0 and Number_of_binding_of_promoter[mol2]==0):
                
                Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=0
                for j in range(Which_aggregate_contains_nuclear_protein[mol1],Number_of_aggregate):
                    Nuclear_protein_in_aggregate[j][:]=Nuclear_protein_in_aggregate[j+1][:]
                    for i in range(Number_of_nuclear_protein_in_aggregate[j+1]):
                            Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]=Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]-1
                    Number_of_nuclear_protein_in_aggregate[j]=Number_of_nuclear_protein_in_aggregate[j+1]
                    
                    Promoter_in_aggregate[j][:]=Promoter_in_aggregate[j+1][:]
                    for i in range(Number_of_promoter_in_aggregate[j+1]):
                            Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]=Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]-1
                    Number_of_promoter_in_aggregate[j]=Number_of_promoter_in_aggregate[j+1]
                    
                    Enhancer_in_aggregate[j][:]=Enhancer_in_aggregate[j+1][:]
                    for i in range(Number_of_enhancer_in_aggregate[j+1]):
                            Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]=Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]-1
                    Number_of_enhancer_in_aggregate[j]=Number_of_enhancer_in_aggregate[j+1]
                    
                Which_aggregate_contains_nuclear_protein[mol1]=-1
                Which_aggregate_contains_promoter[mol2]=-1
                Number_of_aggregate=Number_of_aggregate-1
                
            elif (Number_of_binding_of_nuclear_protein[mol1]==0 and Number_of_binding_of_promoter[mol2]!=0):
            
                bg=0
                while Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][bg]!=mol1:
                        bg=bg+1
                for j in range(bg,Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1):
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j]=Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j+1]
                Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1]=0
                Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1
                Which_aggregate_contains_nuclear_protein[mol1]=-1
                
                nblink=0
                for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]]):
                    nblink=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]][i]]
                retnbpol=0
                for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_promoter[mol2]]):
                    test=np.random.uniform(low=0,high=1,size=None)
                    if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]]-nblink-1+3)):
                        retnbpol=retnbpol+1
                Number_of_PolII_in_aggregate[Which_aggregate_contains_promoter[mol2]]=retnbpol
                
            elif (Number_of_binding_of_nuclear_protein[mol1]!=0 and Number_of_binding_of_promoter[mol2]==0):
            
                bg=0
                while Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][bg]!=mol2:
                        bg=bg+1
                for j in range(bg,Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1):
                    Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j]=Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j+1]
                Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_promoter_in_aggregate[Which_aggregate_contains_promoter[mol2]]-1]=0
                Number_of_promoter_in_aggregate[Which_aggregate_contains_promoter[mol2]]=Number_of_promoter_in_aggregate[Which_aggregate_contains_promoter[mol2]]-1
                Which_aggregate_contains_promoter[mol2]=-1
                
                nblink=0
                for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                    nblink=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][i]]
                retnbpol=0
                for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                    test=np.random.uniform(low=0,high=1,size=None)
                    if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1+3)):
                        retnbpol=retnbpol+1
                Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=retnbpol
                
            #What if the two molecules dissociate also form 2 aggregate
            else:
                chain1TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                chain1Prom=-1*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                chain1Enh=-1*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                
                markchain1TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                markchain1TF[0]=mol1
                markTF=0
                
                sizechain1TF=1
                sizechain1Prom=0
                sizechain1Enh=0
                
                while ((markchain1TF[:]!=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any()):

                    for i in range(Number_of_nuclear_protein):
                        if Molecule_bound_to_nuclear_protein[markchain1TF[markTF],i]==1:
                            if (chain1TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                oposite=0
                            else:
                                markchain1TF[sizechain1TF]=i
                                sizechain1TF=sizechain1TF+1
                                
                    for j in range(Number_promoter):  
                        if Molecule_bound_to_prom[markchain1TF[markTF],j]==1:
                            if (chain1Prom==j*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                opposite=0
                            else:
                                chain1Prom[sizechain1Prom]=j
                                sizechain1Prom=sizechain1Prom+1
                                
                                for p in range(Number_of_nuclear_protein):
                                    if Molecule_bound_to_prom[p,j]==1:
                                        if (chain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                            oposite=0
                                        else:
                                            markchain1TF[sizechain1TF]=p
                                            sizechain1TF=sizechain1TF+1
                    for j in range(Number_enhancer):  
                        if Molecule_next_to_enhancer[markchain1TF[markTF],j]==1:
                            if (chain1Enh==j*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                opposite=0
                            else:
                                chain1Enh[sizechain1Enh]=j
                                sizechain1Enh=sizechain1Enh+1
                                
                                for p in range(Number_of_nuclear_protein):
                                    if Molecule_next_to_enhancer[p,j]==1:
                                        if (chain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                            oposite=0
                                        else:
                                            markchain1TF[sizechain1TF]=p
                                            sizechain1TF=sizechain1TF+1
                                            
                    chain1TF[markTF]=markchain1TF[markTF]
                    markchain1TF[markTF]=-1
                    markTF=markTF+1
                    
                    
                
                if sizechain1TF!=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]:
                    #print('YES WE BREAK FCKING EVERYTHING')
                    chain2TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    chain2Prom=-1*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    chain2Enh=-1*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    
                    markchain2TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    chain2Prom[0]=mol2
                    
                    markTF=0
                    sizechain2TF=0
                    sizechain2Prom=1
                    sizechain2Enh=0
                    
                    #we fullfill once markchain
                    for j in range(Number_of_nuclear_protein):  
                            if Molecule_bound_to_prom[j,mol2]==1:
                                if (markchain2TF==j*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    markchain2TF[sizechain2TF]=j
                                    sizechain2TF=sizechain2TF+1
                    
                    while ((markchain2TF[:]!=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any()):
                        for i in range(Number_of_nuclear_protein):
                            if Molecule_bound_to_nuclear_protein[markchain2TF[markTF],i]==1:
                                if (chain2TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    markchain2TF[sizechain2TF]=i
                                    sizechain2TF=sizechain2TF+1
                        for j in range(Number_promoter):  
                            if Molecule_bound_to_prom[markchain2TF[markTF],j]==1:
                                if (chain2Prom==j*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    chain2Prom[sizechain2Prom]=j
                                    sizechain2Prom=sizechain2Prom+1
                                    
                                    for p in range(Number_of_nuclear_protein):
                                        if Molecule_bound_to_prom[p,j]==1:
                                            if (chain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                                oposite=0
                                            else:
                                                markchain2TF[sizechain2TF]=p
                                                sizechain2TF=sizechain2TF+1
                        for j in range(Number_enhancer):  
                            if Molecule_next_to_enhancer[markchain2TF[markTF],j]==1:
                                if (chain2Enh==j*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    chain2Enh[sizechain2Enh]=j
                                    sizechain2Enh=sizechain2Enh+1
                                    
                                    for p in range(Number_of_nuclear_protein):
                                        if Molecule_next_to_enhancer[p,j]==1:
                                            if (chain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                                oposite=0
                                            else:
                                                markchain2TF[sizechain2TF]=p
                                                sizechain2TF=sizechain2TF+1
                                                
                        chain2TF[markTF]=markchain2TF[markTF]
                        markchain2TF[markTF]=-1
                        markTF=markTF+1
                        
                    Nuclear_protein_in_aggregate[Number_of_aggregate][:sizechain2TF]=chain2TF[:sizechain2TF]
                    Promoter_in_aggregate[Number_of_aggregate][:sizechain2Prom]=chain2Prom[:sizechain2Prom]
                    Enhancer_in_aggregate[Number_of_aggregate][:sizechain2Enh]=chain2Enh[:sizechain2Enh]
                    Number_of_nuclear_protein_in_aggregate[Number_of_aggregate]=sizechain2TF
                    Number_of_promoter_in_aggregate[Number_of_aggregate]=sizechain2Prom
                    Number_of_enhancer_in_aggregate[Number_of_aggregate]=sizechain2Enh
                    
                    for i in range(sizechain2TF):
                        Which_aggregate_contains_nuclear_protein[chain2TF[i]]=Number_of_aggregate
                    for i in range(sizechain2Prom):
                        Which_aggregate_contains_promoter[chain2Prom[i]]=Number_of_aggregate
                    for i in range(sizechain2Enh):
                        Which_aggregate_contains_enhancer[chain2Enh[i]]=Number_of_aggregate
                    Number_of_aggregate=Number_of_aggregate+1
                    
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_of_nuclear_protein)
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1TF]=chain1TF[:sizechain1TF]
                    Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_promoter)
                    Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1Prom]=chain1Prom[:sizechain1Prom]
                    Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_enhancer)
                    Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1Enh]=chain1Enh[:sizechain1Enh]
                    
                    Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2TF
                    Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2Prom
                    Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2Enh
                    
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]:]=np.zeros((Number_of_nuclear_protein-Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]))
                    
                    nblink=0
                    for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                        nblinkmol1=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][i]]
                    for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]]):
                        nblinkmol2=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]][i]]
                        
                    retnbpol=0
                    for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                        test=np.random.uniform(low=0,high=1,size=None)
                        if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblinkmol1+4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_promoter[mol2]]-nblinkmol2-2)):
                            retnbpol=retnbpol+1
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_promoter[mol2]]=Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-retnbpol
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=retnbpol
        
        elif t==Min_Time_to_unbind_nuclear_protein_to_enhancer:
                
            index=np.argmin(Time_to_unbind_nuclear_protein_to_enhancer)
            mol1=index//Number_enhancer
            mol2=index%Number_enhancer
            
            
            
            Number_of_binding_of_nuclear_protein[mol1]=Number_of_binding_of_nuclear_protein[mol1]-1
            Number_of_binding_of_enhancer[mol2]=Number_of_binding_of_enhancer[mol2]-1
            Molecule_next_to_enhancer[mol1,mol2]=0
            
            
            if (Number_of_binding_of_nuclear_protein[mol1]==0 and Number_of_binding_of_enhancer[mol2]==0):
                
                Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]
                for j in range(Which_aggregate_contains_nuclear_protein[mol1],Number_of_aggregate):
                    
                    Nuclear_protein_in_aggregate[j][:]=Nuclear_protein_in_aggregate[j+1][:]
                    for i in range(Number_of_nuclear_protein_in_aggregate[j+1]):
                            Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]=Which_aggregate_contains_nuclear_protein[Nuclear_protein_in_aggregate[j][i]]-1
                    Number_of_nuclear_protein_in_aggregate[j]=Number_of_nuclear_protein_in_aggregate[j+1]
                    
                    Promoter_in_aggregate[j][:]=Promoter_in_aggregate[j+1][:]
                    for i in range(Number_of_promoter_in_aggregate[j+1]):
                            Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]=Which_aggregate_contains_promoter[Promoter_in_aggregate[j][i]]-1
                    Number_of_promoter_in_aggregate[j]=Number_of_promoter_in_aggregate[j+1]
                    
                    Enhancer_in_aggregate[j][:]=Enhancer_in_aggregate[j+1][:]
                    for i in range(Number_of_enhancer_in_aggregate[j+1]):
                            Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]=Which_aggregate_contains_enhancer[Enhancer_in_aggregate[j][i]]-1
                    Number_of_enhancer_in_aggregate[j]=Number_of_enhancer_in_aggregate[j+1]
                    
                Which_aggregate_contains_nuclear_protein[mol1]=-1
                Which_aggregate_contains_enhancer[mol2]=-1
                Number_of_aggregate=Number_of_aggregate-1
                
            elif (Number_of_binding_of_nuclear_protein[mol1]==0 and Number_of_binding_of_enhancer[mol2]!=0):
                
                bg=0
                #we take index of mol1
                while Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][bg]!=mol1:
                        bg=bg+1
                #we modify the aggregate to get rid of mol1 
                for j in range(bg,Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1):
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j]=Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j+1]
                Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1]=0
                Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1
                Which_aggregate_contains_nuclear_protein[mol1]=-1
                
                nblink=0
                for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]]):
                    nblink=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]][i]]
                retnbpol=0
                for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_enhancer[mol2]]):
                    test=np.random.uniform(low=0,high=1,size=None)
                    if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]]-nblink-1+3)):
                        retnbpol=retnbpol+1
                Number_of_PolII_in_aggregate[Which_aggregate_contains_enhancer[mol2]]=retnbpol
                
            elif (Number_of_binding_of_nuclear_protein[mol1]!=0 and Number_of_binding_of_enhancer[mol2]==0):
                
                bg=0
                while Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][bg]!=mol2:
                        bg=bg+1
                for j in range(bg,Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-1):
                    Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j]=Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][j+1]
                Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][Number_of_enhancer_in_aggregate[Which_aggregate_contains_enhancer[mol2]]-1]=0
                Number_of_enhancer_in_aggregate[Which_aggregate_contains_enhancer[mol2]]=Number_of_enhancer_in_aggregate[Which_aggregate_contains_enhancer[mol2]]-1
                Which_aggregate_contains_enhancer[mol2]=-1
                
                nblink=0
                for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                    nblink=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][i]]
                retnbpol=0
                for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                    test=np.random.uniform(low=0,high=1,size=None)
                    if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1+3)):
                        retnbpol=retnbpol+1
                Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=retnbpol
            #What if the two molecules dissociate also form 2 aggregate
            else:
                chain1TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                chain1Prom=-1*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                chain1Enh=-1*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                
                markchain1TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                markchain1TF[0]=mol1
                markTF=0
                
                sizechain1TF=1
                sizechain1Prom=0
                sizechain1Enh=0
                
                while ((markchain1TF[:]!=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any()):
                    #is the marked TF linked to another TF whose not already in chain?
                    for i in range(Number_of_nuclear_protein):
                        if Molecule_bound_to_nuclear_protein[markchain1TF[markTF],i]==1:
                            if (chain1TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                oposite=0
                            else:
                                markchain1TF[sizechain1TF]=i
                                sizechain1TF=sizechain1TF+1
                    #is the marked TF linked to a polymerase
                    for j in range(Number_promoter):  
                        if Molecule_bound_to_prom[markchain1TF[markTF],j]==1:
                            if (chain1Prom==j*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                opposite=0
                            else:
                                chain1Prom[sizechain1Prom]=j
                                sizechain1Prom=sizechain1Prom+1
                                #are those polymerase linked to another TF
                                for p in range(Number_of_nuclear_protein):
                                    if Molecule_bound_to_prom[p,j]==1:
                                        if (chain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                            oposite=0
                                        else:
                                            markchain1TF[sizechain1TF]=p
                                            sizechain1TF=sizechain1TF+1
                    for j in range(Number_enhancer):  
                        if Molecule_next_to_enhancer[markchain1TF[markTF],j]==1:
                            if (chain1Enh==j*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                opposite=0
                            else:
                                chain1Enh[sizechain1Enh]=j
                                sizechain1Enh=sizechain1Enh+1
                                
                                for p in range(Number_of_nuclear_protein):
                                    if Molecule_next_to_enhancer[p,j]==1:
                                        if (chain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain1TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                            oposite=0
                                        else:
                                            markchain1TF[sizechain1TF]=p
                                            sizechain1TF=sizechain1TF+1
                                            
                    chain1TF[markTF]=markchain1TF[markTF]
                    markchain1TF[markTF]=-1
                    markTF=markTF+1
                    
                if sizechain1TF!=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]:
                
                    chain2TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    chain2Prom=-1*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    chain2Enh=-1*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    
                    markchain2TF=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]],int)
                    chain2Enh[0]=mol2
                    
                    markTF=0
                    sizechain2TF=0
                    sizechain2Prom=0
                    sizechain2Enh=1
                    
                    #we fullfill once markchain
                    for j in range(Number_of_nuclear_protein):  
                            if Molecule_next_to_enhancer[j,mol2]==1:
                                if (markchain2TF==j*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    markchain2TF[sizechain2TF]=j
                                    
                                    sizechain2TF=sizechain2TF+1
                    #print("we did the first loop")
                    while ((markchain2TF[:]!=-1*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any()):

                        for i in range(Number_of_nuclear_protein):
                            if Molecule_bound_to_nuclear_protein[markchain2TF[markTF],i]==1:
                                if (chain2TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==i*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    markchain2TF[sizechain2TF]=i
                                    sizechain2TF=sizechain2TF+1
                                    
                        for j in range(Number_promoter):  
                            if Molecule_bound_to_prom[markchain2TF[markTF],j]==1:
                                if (chain2Prom==j*np.ones(Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    chain2Prom[sizechain2Prom]=j
                                    sizechain2Prom=sizechain2Prom+1
                                    
                                    for p in range(Number_of_nuclear_protein):
                                        if Molecule_bound_to_prom[p,j]==1:
                                            if (chain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                                oposite=0
                                            else:
                                                markchain2TF[sizechain2TF]=p
                                                sizechain2TF=sizechain2TF+1
                        for j in range(Number_enhancer):  
                            if Molecule_next_to_enhancer[markchain2TF[markTF],j]==1:
                                if (chain2Enh==j*np.ones(Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                    opposite=0
                                else:
                                    chain2Enh[sizechain2Enh]=j
                                    sizechain2Enh=sizechain2Enh+1
                                    
                                    for p in range(Number_of_nuclear_protein):
                                        if Molecule_next_to_enhancer[p,j]==1:
                                            if (chain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any() or (markchain2TF==p*np.ones(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]])).any():
                                                oposite=0
                                            else:
                                                markchain2TF[sizechain2TF]=p
                                                sizechain2TF=sizechain2TF+1                          
                        chain2TF[markTF]=markchain2TF[markTF]
                        markchain2TF[markTF]=-1
                        markTF=markTF+1
                    Nuclear_protein_in_aggregate[Number_of_aggregate][:sizechain2TF]=chain2TF[:sizechain2TF]
                    Promoter_in_aggregate[Number_of_aggregate][:sizechain2Prom]=chain2Prom[:sizechain2Prom]
                    Enhancer_in_aggregate[Number_of_aggregate][:sizechain2Enh]=chain2Enh[:sizechain2Enh]
                    Number_of_nuclear_protein_in_aggregate[Number_of_aggregate]=sizechain2TF
                    Number_of_promoter_in_aggregate[Number_of_aggregate]=sizechain2Prom
                    Number_of_enhancer_in_aggregate[Number_of_aggregate]=sizechain2Enh
                    
                    for i in range(sizechain2TF):
                        Which_aggregate_contains_nuclear_protein[chain2TF[i]]=Number_of_aggregate
                    for i in range(sizechain2Prom):
                        Which_aggregate_contains_promoter[chain2Prom[i]]=Number_of_aggregate
                    for i in range(sizechain2Enh):
                        Which_aggregate_contains_enhancer[chain2Enh[i]]=Number_of_aggregate
                    Number_of_aggregate=Number_of_aggregate+1
                    
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_of_nuclear_protein)
                    Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1TF]=chain1TF[:sizechain1TF]
                    Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_promoter)
                    Promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1Prom]=chain1Prom[:sizechain1Prom]
                    Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:]=np.zeros(Number_enhancer)
                    Enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][:sizechain1Enh]=chain1Enh[:sizechain1Enh]
                    
                    Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2TF
                    Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_promoter_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2Prom
                    Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=Number_of_enhancer_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-sizechain2Enh
                    
                    nblink=0
                    for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                        nblinkmol1=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]][i]]
                    for i in range(Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]]):
                        nblinkmol2=nblink+Number_of_binding_of_nuclear_protein[Nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]][i]]
                        
                    retnbpol=0
                    for i in range(Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]):
                        test=np.random.uniform(low=0,high=1,size=None)
                        if test<(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblink-1/(4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-nblinkmol1+4*Number_of_nuclear_protein_in_aggregate[Which_aggregate_contains_enhancer[mol2]]-nblinkmol2-2)):
                            retnbpol=retnbpol+1
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_enhancer[mol2]]=Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]-retnbpol
                    Number_of_PolII_in_aggregate[Which_aggregate_contains_nuclear_protein[mol1]]=retnbpol
                        
                            
        mRNAvec[T_index]=mRNA      
        Time_vector[T_index]=t+Time_vector[T_index-1]
        Trajectories_nuclear_protein[T_index,:]=Current_position_nuclear_protein[:]
        Trajectories_promoter[T_index,:]=Current_position_promoter[:]
        Trajectories_enhancer[T_index,:]=Current_position_enhancer[:]
        Number_of_enhancer_in_aggregate_simulation[T_index,:]=Number_of_enhancer_in_aggregate
        Number_of_promoter_in_aggregate_simulation[T_index,:]=Number_of_promoter_in_aggregate
        Number_of_nuclear_protein_in_aggregate_simulation[T_index,:]=Number_of_nuclear_protein_in_aggregate
        Number_of_PolII_in_aggregate_simulation[T_index,:]=Number_of_PolII_in_aggregate
        print(T_index)

        
    return Trajectories_nuclear_protein,Trajectories_promoter,Trajectories_enhancer,Time_vector,Number_of_enhancer_in_aggregate_simulation,Number_of_promoter_in_aggregate_simulation,Number_of_nuclear_protein_in_aggregate_simulation,mRNAvec,Number_of_PolII_in_aggregate_simulation

In [20]:
Number_of_nuclear_protein=400
Number_promoter=5
Current_position_promoter=np.array([2518,2519,2520,2521,2522])
Number_enhancer=15
Current_position_enhancer=np.array([3873,3874,3875,3876,3877,3878,3879,3880,3881,3882,3883,3884,3885,3886,3887])
Num_step=150000
D_nuclear_protein=10
D_promoter=1
D_enhancer=1
nx=80
ny=80
xmin=0
xmax=1
ymin=0
ymax=1
f=4
k_bind_nuclear_protein_to_nuclear_protein=0.1
k_unbinding_nuclear_protein_nuclear_protein=1


k_link_nuclear_protein_promoter=15
k_unbinding_nuclear_protein_promoter=1

k_link_nuclear_protein_enhancer=25
k_unbinding_nuclear_protein_enhancer=1

k_PolII_on_promoter=1
k_PolII_in_aggregate=0.05
k_PolII_out_of_aggregate=0.01
Delta_mRNA=0.5
PolII_per_free_link=1
Number_of_simulation=1

mesh=meshgen(nx,ny,xmin,xmax,ymin,ymax)

#Vector containing the results of several simulations

Size_aggregate_nuclear_protein_multiple_simulation=np.zeros((Num_step,Number_of_nuclear_protein,Number_of_simulation))
Size_aggregate_enhancer_multiple_simulation=np.zeros((Num_step,Number_of_nuclear_protein,Number_of_simulation))
Size_aggregate_promoter_multiple_simulation=np.zeros((Num_step,Number_of_nuclear_protein,Number_of_simulation))

Time_vector_multiple_simulation=np.zeros((Num_step,Number_of_simulation))

mRNA_multiple_simulation=np.zeros((Num_step,Number_of_simulation))
Number_of_PolII_in_aggregate_multiple_simulation=np.zeros((Num_step,Number_of_nuclear_protein,Number_of_simulation))

Trajectories_nuclear_protein_multiple_simulation=np.zeros((Num_step,Number_of_nuclear_protein,Number_of_simulation))
Trajectories_promoter_multiple_simulation=np.zeros((Num_step,Number_promoter,Number_of_simulation))
Trajectories_enhancer_multiple_simulation=np.zeros((Num_step,Number_enhancer,Number_of_simulation))

for i in range(Number_of_simulation):
    Trajectories_nuclear_protein_multiple_simulation[:,:,i],Trajectories_promoter_multiple_simulation[:,:,i],Trajectories_enhancer_multiple_simulation[:,:,i],Time_vector_multiple_simulation[:,i],Size_aggregate_enhancer_multiple_simulation[:,:,i],Size_aggregate_promoter_multiple_simulation[:,:,i],Size_aggregate_nuclear_protein_multiple_simulation[:,:,i],mRNA_multiple_simulation[:,i],Number_of_PolII_in_aggregate_multiple_simulation[:,:,i]=TFDNAdif(PolII_per_free_link,k_PolII_on_promoter,k_PolII_in_aggregate,k_PolII_out_of_aggregate,Num_step,Current_position_enhancer,Current_position_promoter,Number_of_nuclear_protein,Number_promoter,Number_enhancer,Delta_mRNA,mesh,nx,ny,xmin,xmax,ymin,ymax,D_nuclear_protein,D_promoter,D_enhancer,f,k_bind_nuclear_protein_to_nuclear_protein,k_link_nuclear_protein_promoter,k_link_nuclear_protein_enhancer,k_unbinding_nuclear_protein_nuclear_protein,k_unbinding_nuclear_protein_promoter,k_unbinding_nuclear_protein_enhancer)

    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 